In [1]:
import tensorflow as tf
import pickle
import numpy as np

import foolbox
import eagerpy as ep
from tqdm.notebook import tqdm

In [2]:
test_batches = tf.keras.utils.image_dataset_from_directory(
    "../pneumothorax/ChestX-ray14-resized/binary",
    labels='inferred',
    label_mode='categorical',
    color_mode='rgb',
    batch_size=16,
    image_size=(224, 224),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset="validation"
)
test_batches = test_batches.map(lambda x,y: (tf.keras.applications.resnet50.preprocess_input(x),y))

Found 25596 files belonging to 2 classes.
Using 5119 files for validation.


2022-06-28 23:11:11.354004: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-28 23:11:11.381333: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-28 23:11:11.381507: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-28 23:11:11.382359: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [8]:
mins = []
maxs = []
for batch in test_batches:
    mins.append(tf.math.reduce_min(batch[0]))
    maxs.append(tf.math.reduce_max(batch[0]))
print(float(tf.math.reduce_min(mins)), float(tf.math.reduce_max(maxs)))

-123.68000030517578 151.06100463867188


In [3]:
with open("../pneumothorax/Pneumo_model.pkl","rb") as file:
    model_without_attention = pickle.load(file)
with open("../pneumothorax/Pneumo+SA_model.pkl","rb") as file:
    model_with_attention = pickle.load(file)

In [4]:
model = model_with_attention
preprocessing = dict()
bounds = (-123.69, 151.062)
fmodel = foolbox.TensorFlowModel(model, bounds=bounds, preprocessing=preprocessing)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2022-06-28 23:11:32.368743: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-28 23:11:32.368932: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-28 23:11:32.369036: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-28 23:11:32.369175: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-28 23:11:32.369280: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

In [5]:
batch_accs = []
batch_weights = []
for i,(x,y) in zip(range(len(test_batches)),test_batches):
    acc = foolbox.utils.accuracy(fmodel, x, tf.argmax(y, axis=1))
    batch_accs.append(acc)
    batch_weights.append(y.shape[0])
s1 = 0
s2 = sum(batch_weights)
for i in range(len(batch_accs)):
    s1 += batch_accs[i] * batch_weights[i]
print("unperturbed acc:",s1/s2)


2022-06-28 22:39:50.650204: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8400


unperturbed acc: 0.8829849580345407


In [5]:
attack = foolbox.attacks.LinfAdamProjectedGradientDescentAttack()
for model in [model_without_attention, model_with_attention]:
    fmodel = foolbox.TensorFlowModel(model, bounds=bounds, preprocessing=preprocessing)
    for epsilon in [0.00125, 0.0025, 0.005, 0.01, 0.02, 0.04, 0.08, 0.16, 0.32]:
        adv_batches = []
        labels = []
        print("epsilon =", epsilon)
        for i,(x,y) in tqdm(zip(range(len(test_batches)),test_batches), total=len(test_batches)):
            raw, clipped, is_adv = attack(fmodel, tf.constant(x), tf.argmax(y, axis=1), epsilons=[epsilon])
            adv_batches.append(clipped[0])
            labels.append(tf.argmax(y, axis=1))

        batch_accs = []
        batch_weights = []
        for (batch,label) in zip(adv_batches, labels):
            x = np.array(batch)
            y = np.array(label)
            acc = foolbox.utils.accuracy(fmodel, x, y)
            batch_accs.append(acc)
            batch_weights.append(y.shape[0])
        s1 = 0
        s2 = sum(batch_weights)
        for i in range(len(batch_accs)):
            s1 += batch_accs[i] * batch_weights[i]
        print("perturbed acc:",s1/s2)


epsilon = 0.00125


2022-06-28 23:12:25.495577: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-28 23:12:25.495873: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-28 23:12:25.495999: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-28 23:12:25.496162: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-28 23:12:25.496391: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

  0%|          | 0/320 [00:00<?, ?it/s]

2022-06-28 23:12:26.113164: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8400


perturbed acc: 0.8972455558075433
epsilon = 0.0025


...

perturbed acc: 0.8704825161397166
epsilon = 0.16


  0%|          | 0/320 [00:00<?, ?it/s]

perturbed acc: 0.8534870092164122
epsilon = 0.32


  0%|          | 0/320 [00:00<?, ?it/s]

perturbed acc: 0.8052353975618693


Using `FGSM()`
| Model/Epsilon     | 0                   | 0.00125             | 0.0025              | 0.005               | 0.01                | 0.02                | 0.04                | 0.08               | 0.16                | 0.32                |
| ----------------- | ------------------- | ------------------- | ------------------- | ------------------- | ------------------- | ------------------- | ------------------- |------------------- | ------------------- | ------------------- |
| Without Attention | 0.8972455557958995  | 0.8972455557726119  | 0.8972455557958995  | 0.8964641531782007  | 0.8952920492400087  | 0.8931431920762686  | 0.8884547763467883  | 0.8792732955771839 | 0.8587614769091665  | 0.786872436057592   |
| With Attention    | 0.8829849580345407  | 0.8827896073451846  | 0.8825942567024038  | 0.8823989060596228  | 0.8816175034535678  | 0.879859347540458   | 0.8763430357608135  | 0.868333658941045  | 0.8472357882631782  | 0.792146903715415   |

Using `LinfAdamProjectedGradientDescentAttack()`
| Model/Epsilon     | 0                   | 0.00125             | 0.0025              | 0.005               | 0.01                | 0.02                | 0.04                | 0.08                | 0.16                | 0.32                |
| ----------------- | ------------------- | ------------------- | ------------------- | ------------------- | ------------------- | ------------------- | ------------------- | ------------------- | ------------------- | ------------------- |
| Without Attention | 0.8972455557958995  | 0.8972455558075433  | 0.8972455557726119  | 0.8964641531549131  | 0.895682750560502   | 0.8939245946706796  | 0.8892361789528432  | 0.8814221527874994  | 0.8636452432232086  | 0.8048446962646638  |
| With Attention    | 0.8829849580345407  | 0.8827896073684722  | 0.8825942567024038  | 0.8823989060829105  | 0.8820082047391297  | 0.8800546981948827  | 0.8771244383901562  | 0.8704825161397166  | 0.8534870092164122  | 0.8052353975618693  |
